In [1]:
import dropbox
import pandas as pd
import numpy as np
import datetime
import time
import pytz
import os

In [2]:
client = dropbox.client.DropboxClient('IS-424yqxy8AAAAAAAAVLOUS9urGIH4kCxP_5Q6hxdz-WrhGMYKa-9MjMZrpwMYZ')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: You are using a deprecated client. Please use the new v2 client located at dropbox.Dropbox.
  if __name__ == '__main__':


In [4]:
metadata = client.metadata('/')

In [20]:
for i in metadata:
    print(i)

read_only
hash
thumb_exists
bytes
path
is_dir
icon
root
contents
size


In [45]:
for i in metadata['contents']:
    print (i['modified'])

Fri, 05 May 2017 23:22:03 +0000
Fri, 05 May 2017 23:22:06 +0000
Fri, 05 May 2017 23:22:12 +0000
Fri, 05 May 2017 23:22:17 +0000
Fri, 05 May 2017 23:22:23 +0000
Fri, 05 May 2017 23:22:26 +0000
Fri, 05 May 2017 23:22:34 +0000
Fri, 05 May 2017 23:22:39 +0000
Fri, 05 May 2017 23:22:43 +0000
Fri, 05 May 2017 23:22:49 +0000
Fri, 05 May 2017 23:22:54 +0000
Fri, 05 May 2017 23:23:00 +0000
Fri, 05 May 2017 23:23:06 +0000
Fri, 05 May 2017 23:23:12 +0000
Fri, 05 May 2017 23:23:18 +0000
Fri, 05 May 2017 23:23:22 +0000
Fri, 05 May 2017 23:23:30 +0000
Fri, 05 May 2017 23:23:40 +0000
Fri, 05 May 2017 23:23:50 +0000
Fri, 05 May 2017 23:23:58 +0000
Fri, 05 May 2017 23:24:05 +0000
Fri, 05 May 2017 23:24:17 +0000
Fri, 05 May 2017 23:24:21 +0000
Fri, 05 May 2017 23:24:25 +0000
Fri, 05 May 2017 23:24:32 +0000
Fri, 05 May 2017 23:24:40 +0000
Fri, 05 May 2017 23:24:45 +0000
Fri, 05 May 2017 23:24:47 +0000
Fri, 05 May 2017 23:24:50 +0000
Fri, 05 May 2017 23:24:53 +0000
Fri, 05 May 2017 23:24:56 +0000
Fri, 05 

In [3]:
metadata = client.metadata('/')
files = [content['path'].split('/')[-1] for content in metadata['contents'] 
         if content['is_dir'] == False &
         datetime.datetime(*time.strptime(content['modified'], "%a, %d %b %Y %H:%M:%S %z")[:6]).date() == datetime.datetime.today().date()] 

MaxRetryError: HTTPSConnectionPool(host='api.dropbox.com', port=443): Max retries exceeded with url: /1/metadata/auto?file_limit=25000&list=true&include_deleted=False&include_media_info=False (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000021F24977860>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))

In [31]:
client.share('/ABA DUENAS_0000.JPG', short_url=False)['url'][:-1] + '1'

'https://www.dropbox.com/s/15bgbss3xzkvjlq/ABA%20DUENAS_0000.JPG?dl=1'

In [22]:
datetime.datetime(*time.localtime()[:6])

datetime.datetime(2017, 5, 7, 10, 37, 55)

In [2]:
def files():
    client = dropbox.client.DropboxClient('IS-424yqxy8AAAAAAAAVLOUS9urGIH4kCxP_5Q6hxdz-WrhGMYKa-9MjMZrpwMYZ')
    metadata = client.metadata('/')
    files = [content['path'].split('/')[-1] for content in metadata['contents'] if content['is_dir'] == False]
    name = [i[0:i.find('_')] for i in files]
    files = pd.DataFrame({'file_name': files, 'c_name': name})
    dates = [content['modified'] for content in metadata['contents'] if content['is_dir'] == False]
    files['date_mod_d'] = [time.strptime(i, "%a, %d %b %Y %H:%M:%S %z") for i in dates]
    files['date_mod'] = [datetime.datetime(*files.date_mod_d.loc[i][:6]).date() for i in files.index]
    files['hour_mod'] = [datetime.datetime(*files.date_mod_d.loc[i][:6]).time() for i in files.index]
    del files['date_mod_d']
    files['check1'] = np.nan
    files['check2'] = np.nan
    return files   

In [10]:
def dl_files(x):
    client = dropbox.client.DropboxClient('IS-424yqxy8AAAAAAAAVLOUS9urGIH4kCxP_5Q6hxdz-WrhGMYKa-9MjMZrpwMYZ')
    df = pd.DataFrame(x.groupby('c_name').c_name.count())
    files = x
    for i in df.index:
        c_dir = 'Pancitas Ultrasound {}'.format(i.lower().title())
        os.mkdir(c_dir)
        os.chdir('.\\' + c_dir)
        dl_df = []
        for k in files.file_name:
            if i in k:
                dl_df.append(k)
        for j in dl_df:
            f, metadata = client.get_file_and_metadata('/'+j)
            out = open(j, 'wb')
            out.write(f.read())
            out.close()
        os.chdir('..\\')
        
        
    return 
dl_files(files())

In [11]:
def ul_files():
    dirlst = next(os.walk('.'))[1]
    links = []
    for i in dirlst:
        if "Pancitas" in i:
            db_path = '/' + i
            cwd = '.\\' + i
            os.chdir(cwd)
            filist = os.dirlist(cwd)
            #Watermark function here
            for k in filist:
                if 'Pancitas' in i:
                    f = open(k, 'rb')
                    response = client.put_file(db_path + i, f)
                else:
                    os.remove(i)
            link = client.share(db_path, short_url = False)[0] + '1'
            links.append([i, link])
    return links


datetime.date(2017, 5, 17)

In [57]:
def files():
    client = dropbox.client.DropboxClient('IS-424yqxy8AAAAAAAAVLOUS9urGIH4kCxP_5Q6hxdz-WrhGMYKa-9MjMZrpwMYZ')
    metadata = client.metadata('/')
    
    files = [content['path'].split('/')[-1] for content in metadata['contents'] if content['is_dir'] == False]
    size = [round(content['bytes']/10**6, 2) for content in metadata['contents'] if content['is_dir'] == False]
    dates = [content['modified'] for content in metadata['contents'] if content['is_dir'] == False]
    
    name = [i[0:i.find('_')] for i in files]
    files = pd.DataFrame({'file_name': files, 'c_name': name, 'size': size, 'dates': dates})
    
    files['date_mod_d'] = [time.strptime(i, "%a, %d %b %Y %H:%M:%S %z") for i in dates]
    files['date_mod_d'] = [datetime.datetime(*files.date_mod_d.loc[i][:6]) - datetime.timedelta(hours = 4) for i in files.index]
    files['date_mod'] = [files.loc[i].date_mod_d.date() for i in files.index]
    files['hour_mod'] = [files.loc[i].date_mod_d.time() for i in files.index]
    #tday = datetime.datetime.today().date()
    #files = files[files.date_mod == tday]
    
    del files['date_mod_d'], files['dates']
    files['check1'] = np.nan
    files['check2'] = np.nan
    return files   

files()



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: DeprecationWarning: You are using a deprecated client. Please use the new v2 client located at dropbox.Dropbox.
  from ipykernel import kernelapp as app


,c_name,file_name,size,date_mod,hour_mod,check1,check2
0,ABA DUENAS,ABA DUENAS_0001.JPG,0.12,2017-05-18,10:44:25,NaN,NaN
1,ABA DUENAS,ABA DUENAS_0005.JPG,0.12,2017-05-17,19:26:31,NaN,NaN
2,ABA DUENAS,ABA DUENAS_0009.JPG,0.12,2017-05-17,19:26:31,NaN,NaN
3,ADRIANA GRAELL WK27,ADRIANA GRAELL WK27_0001.JPG,0.21,2017-05-12,16:03:13,NaN,NaN
4,ADRIANA GRAELL WK27,ADRIANA GRAELL WK27_0013.JPG,0.07,2017-05-12,16:03:15,NaN,NaN
5,ADRIANA GRAELL WK27,ADRIANA GRAELL WK27_0013.MPG,0.23,2017-05-12,16:03:21,NaN,NaN
6,ADRIANA GRAELL WK27,ADRIANA GRAELL WK27_0023.JPG,0.07,2017-05-12,16:03:23,NaN,NaN
7,ADRIANA GRAELL WK27,ADRIANA GRAELL WK27_0024.JPG,0.07,2017-05-12,16:03:26,NaN,NaN
8,ADRIANA GRAELL WK27,ADRIANA GRAELL WK27_0028.JPG,0.23,2017-05-12,16:03:32,NaN,NaN
9,ADRIANA GRAELL WK27,ADRIANA GRAELL WK27_0033.JPG,0.23,2017-05-12,16:03:37,NaN,NaN


In [87]:
a = pd.DataFrame({'a': [1,2,3,4], 'b' : ['a', 'b', 'c', 'd']})
b = pd.DataFrame({'c': [1,2,5,6], 'd':['a', 'b','e', 'f']})
c = [i not in b.c for i in a.a]
c      
def match_nonmatch(List1, List2):
    List3 = []
    for i in range(len(List1)):
        for j in range(len(List2)):
            if List1[i] == List2[j]:
                List3.append(True)
                break             # fix #1
        else:                     # fix #2
            List3.append(False)
    return List3

match_nonmatch(a.a,b.c)

def updat():
    
    return files()[match_nonmatch(files().file_name, files().file_name)]
updat()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: DeprecationWarning: You are using a deprecated client. Please use the new v2 client located at dropbox.Dropbox.
  from ipykernel import kernelapp as app


,c_name,file_name,size,date_mod,hour_mod,check1,check2
0,ABA DUENAS,ABA DUENAS_0001.JPG,0.12,2017-05-18,10:44:25,NaN,NaN
1,ABA DUENAS,ABA DUENAS_0005.JPG,0.12,2017-05-17,19:26:31,NaN,NaN
2,ABA DUENAS,ABA DUENAS_0009.JPG,0.12,2017-05-17,19:26:31,NaN,NaN
3,ADRIANA GRAELL WK27,ADRIANA GRAELL WK27_0001.JPG,0.21,2017-05-12,16:03:13,NaN,NaN
4,ADRIANA GRAELL WK27,ADRIANA GRAELL WK27_0013.JPG,0.07,2017-05-12,16:03:15,NaN,NaN
5,ADRIANA GRAELL WK27,ADRIANA GRAELL WK27_0013.MPG,0.23,2017-05-12,16:03:21,NaN,NaN
6,ADRIANA GRAELL WK27,ADRIANA GRAELL WK27_0023.JPG,0.07,2017-05-12,16:03:23,NaN,NaN
7,ADRIANA GRAELL WK27,ADRIANA GRAELL WK27_0024.JPG,0.07,2017-05-12,16:03:26,NaN,NaN
8,ADRIANA GRAELL WK27,ADRIANA GRAELL WK27_0028.JPG,0.23,2017-05-12,16:03:32,NaN,NaN
9,ADRIANA GRAELL WK27,ADRIANA GRAELL WK27_0033.JPG,0.23,2017-05-12,16:03:37,NaN,NaN
